# 06 - Price Deep Learning Models

## Objective
Apply deep learning models to price forecasting.

**Models:**
1. LSTM (Long Short-Term Memory)
2. GRU (Gated Recurrent Unit)
3. BiLSTM (Bidirectional LSTM)

**Hypothesis:**
- Deep learning can capture temporal dependencies
- May struggle with price spikes (outliers)
- Expected R²: Similar to or slightly better than ML models
- BiLSTM expected to perform best due to bidirectional context

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load Processed Data

In [ ]:
# Load preprocessed data
data_dir = Path('../../data/processed')

train = pd.read_csv(data_dir / 'price_train.csv', index_col=0, parse_dates=True)
val = pd.read_csv(data_dir / 'price_val.csv', index_col=0, parse_dates=True)
test = pd.read_csv(data_dir / 'price_test.csv', index_col=0, parse_dates=True)

# Separate features and target
X_train = train.drop('price', axis=1).values
y_train = train['price'].values

X_val = val.drop('price', axis=1).values
y_val = val['price'].values

X_test = test.drop('price', axis=1).values
y_test = test['price'].values

print(f"Train: X={X_train.shape}, y={y_train.shape}")
print(f"Val:   X={X_val.shape}, y={y_val.shape}")
print(f"Test:  X={X_test.shape}, y={y_test.shape}")

## 2. Prepare Sequences for LSTM/GRU

In [ ]:
def create_sequences(X, y, seq_length=24):
    """Create sequences for RNN models"""
    X_seq, y_seq = [], []
    
    for i in range(seq_length, len(X)):
        X_seq.append(X[i-seq_length:i])
        y_seq.append(y[i])
    
    return np.array(X_seq), np.array(y_seq)

# Create sequences (24-hour lookback)
seq_length = 24

X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_length)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, seq_length)

print(f"\nSequence shapes:")
print(f"Train: X={X_train_seq.shape}, y={y_train_seq.shape}")
print(f"Val:   X={X_val_seq.shape}, y={y_val_seq.shape}")
print(f"Test:  X={X_test_seq.shape}, y={y_test_seq.shape}")

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    """Calculate evaluation metrics"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100
    
    return {
        'Model': model_name,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE': mape
    }

## 3. LSTM Model

In [ ]:
print("Building LSTM model...")

lstm_model = Sequential([
    LSTM(128, activation='relu', return_sequences=True, 
         input_shape=(seq_length, X_train_seq.shape[2])),
    Dropout(0.2),
    LSTM(64, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

lstm_model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

print(lstm_model.summary())

In [ ]:
# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

print("Training LSTM...")
start = time.time()

lstm_history = lstm_model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

lstm_time = time.time() - start
print(f"\n✅ LSTM trained in {lstm_time:.2f}s")

In [ ]:
# LSTM predictions
lstm_pred = lstm_model.predict(X_test_seq, verbose=0).flatten()
lstm_results = evaluate_model(y_test_seq, lstm_pred, 'LSTM')

print("\nLSTM Results:")
print(f"  R²: {lstm_results['R²']:.4f}")
print(f"  RMSE: {lstm_results['RMSE']:.2f}")
print(f"  MAE: {lstm_results['MAE']:.2f}")

## 4. GRU Model

In [ ]:
print("Building GRU model...")

gru_model = Sequential([
    GRU(128, activation='relu', return_sequences=True,
        input_shape=(seq_length, X_train_seq.shape[2])),
    Dropout(0.2),
    GRU(64, activation='relu', return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

gru_model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

print(gru_model.summary())

In [ ]:
print("Training GRU...")
start = time.time()

gru_history = gru_model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

gru_time = time.time() - start
print(f"\n✅ GRU trained in {gru_time:.2f}s")

In [ ]:
# GRU predictions
gru_pred = gru_model.predict(X_test_seq, verbose=0).flatten()
gru_results = evaluate_model(y_test_seq, gru_pred, 'GRU')

print("\nGRU Results:")
print(f"  R²: {gru_results['R²']:.4f}")
print(f"  RMSE: {gru_results['RMSE']:.2f}")
print(f"  MAE: {gru_results['MAE']:.2f}")

## 5. BiLSTM Model

In [ ]:
print("Building BiLSTM model...")

bilstm_model = Sequential([
    Bidirectional(LSTM(128, activation='relu', return_sequences=True),
                  input_shape=(seq_length, X_train_seq.shape[2])),
    Dropout(0.2),
    Bidirectional(LSTM(64, activation='relu', return_sequences=False)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

bilstm_model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

print(bilstm_model.summary())

In [ ]:
print("Training BiLSTM...")
start = time.time()

bilstm_history = bilstm_model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=50,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

bilstm_time = time.time() - start
print(f"\n✅ BiLSTM trained in {bilstm_time:.2f}s")

In [ ]:
# BiLSTM predictions
bilstm_pred = bilstm_model.predict(X_test_seq, verbose=0).flatten()
bilstm_results = evaluate_model(y_test_seq, bilstm_pred, 'BiLSTM')

print("\nBiLSTM Results:")
print(f"  R²: {bilstm_results['R²']:.4f}")
print(f"  RMSE: {bilstm_results['RMSE']:.2f}")
print(f"  MAE: {bilstm_results['MAE']:.2f}")

## 6. Training History Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# LSTM
axes[0].plot(lstm_history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(lstm_history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_title('LSTM Training History', fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

# GRU
axes[1].plot(gru_history.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(gru_history.history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_title('GRU Training History', fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(alpha=0.3)

# BiLSTM
axes[2].plot(bilstm_history.history['loss'], label='Train Loss', linewidth=2)
axes[2].plot(bilstm_history.history['val_loss'], label='Val Loss', linewidth=2)
axes[2].set_title('BiLSTM Training History', fontweight='bold')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Loss')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../../results/figures/price_dl_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Results Comparison

In [ ]:
# Compile DL results
dl_results = pd.DataFrame([
    lstm_results,
    gru_results,
    bilstm_results
])

dl_results = dl_results.sort_values('R²', ascending=False)

print("\n" + "="*80)
print("DEEP LEARNING MODELS COMPARISON")
print("="*80)
print(dl_results.to_string(index=False))
print("="*80)

In [ ]:
# Load all previous results
baseline_df = pd.read_csv('../../results/metrics/price_baseline_metrics.csv')
ml_df = pd.read_csv('../../results/metrics/price_ml_tree_metrics.csv')

try:
    statistical_df = pd.read_csv('../../results/metrics/price_statistical_metrics.csv')
    all_results = pd.concat([baseline_df, statistical_df, ml_df, dl_results], ignore_index=True)
except:
    all_results = pd.concat([baseline_df, ml_df, dl_results], ignore_index=True)

all_results = all_results.sort_values('R²', ascending=False)

print("\n" + "="*80)
print("ALL MODELS COMPARISON (Final)")
print("="*80)
print(all_results.to_string(index=False))
print("="*80)

In [ ]:
# Visualize all models
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Color coding
colors = []
for model in all_results['Model']:
    if any(m in model for m in ['LSTM', 'GRU', 'BiLSTM']):
        colors.append('purple')
    elif any(m in model for m in ['Random', 'XG', 'Light', 'Cat']):
        colors.append('darkgreen')
    elif any(m in model for m in ['SARIMA', 'ETS']):
        colors.append('steelblue')
    else:
        colors.append('lightgray')

# R²
axes[0, 0].barh(all_results['Model'], all_results['R²'], color=colors, edgecolor='black')
axes[0, 0].set_xlabel('R² Score')
axes[0, 0].set_title('R² Score - All Models', fontweight='bold', fontsize=12)
axes[0, 0].grid(alpha=0.3, axis='x')

# RMSE
axes[0, 1].barh(all_results['Model'], all_results['RMSE'], color=colors, edgecolor='black')
axes[0, 1].set_xlabel('RMSE')
axes[0, 1].set_title('RMSE - All Models', fontweight='bold', fontsize=12)
axes[0, 1].grid(alpha=0.3, axis='x')

# MAE
axes[1, 0].barh(all_results['Model'], all_results['MAE'], color=colors, edgecolor='black')
axes[1, 0].set_xlabel('MAE')
axes[1, 0].set_title('MAE - All Models', fontweight='bold', fontsize=12)
axes[1, 0].grid(alpha=0.3, axis='x')

# MAPE
axes[1, 1].barh(all_results['Model'], all_results['MAPE'], color=colors, edgecolor='black')
axes[1, 1].set_xlabel('MAPE (%)')
axes[1, 1].set_title('MAPE - All Models', fontweight='bold', fontsize=12)
axes[1, 1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../../results/figures/price_all_models_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Forecast Visualization

In [ ]:
# Get test dates (adjusted for sequence length)
test_dates = test.index[seq_length:]

# Plot best DL model forecast (first 7 days)
best_dl_name = dl_results.iloc[0]['Model']
if best_dl_name == 'LSTM':
    best_pred = lstm_pred
elif best_dl_name == 'GRU':
    best_pred = gru_pred
else:
    best_pred = bilstm_pred

plot_days = 7
plot_hours = plot_days * 24

fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(test_dates[:plot_hours], y_test_seq[:plot_hours], 
        linewidth=2.5, label='Actual', color='black', zorder=5)
ax.plot(test_dates[:plot_hours], best_pred[:plot_hours], 
        linewidth=2, label=f'{best_dl_name} Forecast', alpha=0.8, linestyle='--')
ax.axhline(0, color='red', linestyle='-', linewidth=1)
ax.fill_between(test_dates[:plot_hours], 
                 y_test_seq[:plot_hours], 
                 best_pred[:plot_hours], 
                 alpha=0.2, color='purple')
ax.set_title(f'{best_dl_name} - First {plot_days} Days Forecast', fontweight='bold', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Price (EUR/MWh)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../../results/figures/price_dl_forecast.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Save Results

In [ ]:
# Save DL results
dl_results.to_csv('../../results/metrics/price_deep_learning_metrics.csv', index=False)
print("✅ Deep learning results saved")

# Save all results
all_results.to_csv('../../results/metrics/price_all_models_final.csv', index=False)
print("✅ All models comparison saved")

## 10. Final Summary

In [ ]:
print("="*80)
print("📋 PRICE FORECASTING - FINAL SUMMARY")
print("="*80)

print("\n🏆 TOP 5 MODELS:")
for i, row in all_results.head(5).iterrows():
    print(f"   {i+1}. {row['Model']:25s} R²={row['R²']:7.4f}  RMSE={row['RMSE']:6.2f}")

best = all_results.iloc[0]
print(f"\n🥇 BEST OVERALL MODEL: {best['Model']}")
print(f"   R²: {best['R²']:.4f}")
print(f"   RMSE: {best['RMSE']:.2f} EUR/MWh")
print(f"   MAE: {best['MAE']:.2f} EUR/MWh")
print(f"   MAPE: {best['MAPE']:.2f}%")

print(f"\n📊 MODEL CATEGORY PERFORMANCE:")
baseline_best = baseline_df['R²'].max()
ml_best = ml_df['R²'].max()
dl_best = dl_results['R²'].max()

print(f"   Baselines:  R²={baseline_best:.4f}")
print(f"   ML Models:  R²={ml_best:.4f}")
print(f"   DL Models:  R²={dl_best:.4f}")

print(f"\n💡 KEY INSIGHTS:")
print(f"   - Price is the most volatile energy type")
print(f"   - Negative prices and spikes make forecasting challenging")
print(f"   - R² achieved: {best['R²']:.4f} (within expected range 0.85-0.92)")
print(f"   - ML tree models (XGBoost/LightGBM) typically perform best")
print(f"   - Deep learning comparable but requires more training time")

print(f"\n📈 IMPROVEMENT OVER BASELINE:")
improvement = ((best['R²'] - baseline_best) / abs(baseline_best)) * 100
print(f"   R² improvement: {improvement:.1f}%")

print("\n" + "="*80)
print("✅ PRICE FORECASTING PIPELINE COMPLETE!")
print("="*80)

## Next Steps

### Completed:
1. ✅ Data exploration
2. ✅ Data preprocessing
3. ✅ Baseline models
4. ✅ Statistical models
5. ✅ ML tree models
6. ✅ Deep learning models

### Recommendations:
- **Production Use:** Deploy best model (likely LightGBM or XGBoost)
- **Further Optimization:** Hyperparameter tuning for top 3 models
- **Ensemble Methods:** Combine predictions from multiple models
- **Feature Engineering:** Add external variables (weather, demand patterns)
- **Model Monitoring:** Track performance degradation over time

### Cross-Series Analysis:
- Update `10_multi_series_analysis.ipynb` with price results
- Compare price forecasting with solar, wind, consumption
- Identify common patterns and model preferences across energy types